In [1]:
import pandas as pd
import pyranges as pr
import upsetplot
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys

p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

## What % of expressed protein-coding genes do we detected >1 isoform for? 

In [3]:
h5 = '../cerberus_annot.h5'
filt_ab = '../cerberus_filtered_abundance.tsv'
feat = 'tss'

In [4]:
df = pd.read_csv(filt_ab, sep='\t')
df, tids = get_tpm_table(df,
               how='iso',
               min_tpm=1,
               gene_subset='polya',
               h5=h5)

Calculating iso TPM values
Subsetting for polya genes


/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/scripts/utils.py:1805: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tpm_col] = (df[d]*1000000)/df[total_col]
/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/scripts/utils.py:1804: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[total_col] = df[d].sum()


Enforcing minimum TPM
Total # isos detected: 234040
# isos >= 1 tpm: 212911
Applying gene type and novelty subset
Number of isos reported: 209476


In [5]:
ca = cerberus.read(h5)

In [7]:
source = 'lapa'
df = ca.t_map.loc[ca.t_map.source == source].copy(deep=True)

In [10]:
print(len(df.index))
print(len(df.transcript_id.unique().tolist()))

322461
234066


In [16]:
# limit to detected isoforms
df = df.loc[df.transcript_id.isin(tids)]
print(len(df.index))
print(len(df.transcript_id.unique().tolist()))
df = df[['gene_id', 'transcript_id']].drop_duplicates()
print(len(df.index))


297207
209476
209476


In [19]:
df = df.groupby('gene_id').count().reset_index()
df.rename({'transcript_id': 'n_isos'}, axis=1, inplace=True)

In [24]:
gene_df, _, _ = get_gtf_info(how='gene', ver='v40_cerberus', add_stable_gid=True)
gene_df.head()
df = df.merge(gene_df, how='left', left_on='gene_id', right_on='gid_stable')

In [26]:
# only query PC genes
df = df.loc[df.biotype=='protein_coding']

In [27]:
df.head()

,gene_id,n_isos,gid,gname,length,biotype,biotype_category,tf,gid_stable
0,ENSG00000000003,28,ENSG00000000003.15,TSPAN6,12984,protein_coding,protein_coding,False,ENSG00000000003
1,ENSG00000000005,1,ENSG00000000005.6,TNMD,15050,protein_coding,protein_coding,False,ENSG00000000005
2,ENSG00000000419,15,ENSG00000000419.14,DPM1,24374,protein_coding,protein_coding,False,ENSG00000000419
3,ENSG00000000457,7,ENSG00000000457.14,SCYL3,44737,protein_coding,protein_coding,False,ENSG00000000457
4,ENSG00000000460,15,ENSG00000000460.17,C1orf112,192174,protein_coding,protein_coding,False,ENSG00000000460


In [28]:
n_multiple = len(df.loc[df.n_isos>1].index)
n = len(df.index)
print('{:.2f}% of protein-coding genes ({}/{}) have >1 isoform detected'.format((n_multiple/n)*100,n_multiple,n))

89.82% of protein-coding genes (15740/17523) have >1 isoform detected
